In [0]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix


np.random.seed(42)

Using TensorFlow backend.


In [0]:
drive.mount('/content/Drive/',force_remount =  True)

Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/Final Data/"

In [0]:
#Loading training pickle files
sound_lists_pkl_in = open(dirpath+"ca_data_padded_tr.pkl", "rb")
ca_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"en_data_padded_tr.pkl", "rb")
en_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"fr_data_padded_tr.pkl", "rb")
fr_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"it_data_padded_tr.pkl", "rb")
it_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"de_data_padded_tr.pkl", "rb")
de_data_tr = pickle.load(sound_lists_pkl_in)



In [0]:
#Loading test pickle files
sound_lists_pkl_in = open(dirpath+"ca_data_padded_te.pkl", "rb")
ca_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"en_data_padded_te.pkl", "rb")
en_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"fr_data_padded_te.pkl", "rb")
fr_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"it_data_padded_te.pkl", "rb")
it_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"de_data_padded_te.pkl", "rb")
de_data_te = pickle.load(sound_lists_pkl_in)

In [0]:
final_data_tr = np.concatenate((ca_data_tr, en_data_tr, fr_data_tr, it_data_tr, de_data_tr), axis = 0)

In [0]:
final_data_te = np.concatenate((ca_data_te, en_data_te, fr_data_te, it_data_te, de_data_te), axis = 0)

In [0]:
print(final_data_tr.shape)
print(final_data_te.shape)

(10000, 2)
(2500, 2)


In [0]:
del(de_data_te)
del(ca_data_te)
del(en_data_te)
del(fr_data_te)
del(it_data_te)

In [0]:
del(de_data_tr)
del(ca_data_tr)
del(en_data_tr)
del(fr_data_tr)
del(it_data_tr)

In [0]:
# final_data_tr

In [0]:
np.random.shuffle(final_data_tr)
np.random.shuffle(final_data_te)

In [0]:
# data_padded_pkl = open(dirpath+"final_data_tr.pkl", "wb")
# pickle.dump(final_data_tr, data_padded_pkl)

In [0]:
# data_padded_pkl = open(dirpath+"final_data_te.pkl", "wb")
# pickle.dump(final_data_te, data_padded_pkl)

### Labels

In [0]:

labels_tr =[]
for i in range(len(final_data_tr)):

      labels_tr.append(final_data_tr[i][1])  

labels_tr = np.array(labels_tr)
labels_tr

classnames, indices = np.unique(labels_tr, return_inverse=True)
print(indices)

final_labels_tr = indices

[4 2 0 ... 2 0 4]


In [0]:
labels_te =[]
for i in range(len(final_data_te)):

      labels_te.append(final_data_te[i][1])  

labels_te = np.array(labels_te)
labels_te

classnames, indices = np.unique(labels_te, return_inverse=True)
print(indices)

final_labels_te = indices

[3 2 1 ... 1 3 3]


In [0]:
final_X_tr =[]
for i in range(len(final_data_tr)):

      final_X_tr.append(final_data_tr[i][0])  

final_X_tr = np.array(final_X_tr)
final_X_tr[0].shape




(972, 40)

In [0]:
final_X_te =[]
for i in range(len(final_data_te)):

      final_X_te.append(final_data_te[i][0])  

final_X_te = np.array(final_X_te)
final_X_te[0].shape


(972, 40)

### Keras Model

In [0]:
from keras.layers.core import Dense, Permute, Reshape, Flatten 
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical

In [0]:
model = Sequential()
weight_decay = 0.001

model.add(Convolution2D(32, kernel_size=(4,4), strides=(1,1), activation="relu", padding="same", kernel_regularizer=l2(weight_decay), input_shape=(972,40,1)))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(64, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))



model.add(Convolution2D(128, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(256, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

model.add(Convolution2D(512, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(512, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

print(model.layers[-1].output_shape)

#model.add(Permute((2,1,3)))

print(model.layers[-1].output_shape)

bs, x, y, c = model.layers[-1].output_shape

model.add(Reshape([x,y*c]))

print(model.layers[-1].output_shape)

#model.add(LSTM(256, activation="tanh", return_sequences=True))

model.add(LSTM(512, activation="relu",dropout = 0.7, return_sequences=False))

# model.add(Flatten())

# model.add(Dense(256, activation="relu"))

# model.add(Dropout(rate=0.7))

model.add(Dense(5, activation="softmax"))

(None, 16, 1, 512)
(None, 16, 1, 512)
(None, 16, 512)


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 972, 40, 32)       544       
_________________________________________________________________
batch_normalization_71 (Batc (None, 972, 40, 32)       128       
_________________________________________________________________
max_pooling2d_71 (MaxPooling (None, 486, 20, 32)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 486, 20, 64)       32832     
_________________________________________________________________
batch_normalization_72 (Batc (None, 486, 20, 64)       256       
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 243, 10, 64)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 243, 10, 128)      131200    
__________

In [0]:
# model = Sequential()
# weight_decay = 0.001
# model.add(Convolution2D(64, 2, strides = 1, padding='same',W_regularizer=l2(weight_decay), activation="relu", input_shape=(972,40,1)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(10,5), strides=(2, 2)))

# print(model.layers[-1].output_shape)

# model.add(Convolution2D(128, 2,  strides = 1, padding='same', W_regularizer=l2(weight_decay), activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(10,5), strides=(2, 2)))

# print(model.layers[-1].output_shape)

# model.add(Convolution2D(256, 2,  strides = 1, padding='same', W_regularizer=l2(weight_decay), activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(10,2), strides=(2, 2)))

# print(model.layers[-1].output_shape)

# model.add(Convolution2D(256, 2,  strides = 1, padding='same', W_regularizer=l2(weight_decay), activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(10,2), strides=(2, 2)))

# print(model.layers[-1].output_shape)

# # model.add(Convolution2D(512, 2,  strides = 1, padding='same', W_regularizer=l2(weight_decay), activation="relu"))
# # model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(10,2), strides=(2, 2)))

# # print(model.layers[-1].output_shape)

# # model.add(Convolution2D(512, 2,  strides = 1, padding='same', W_regularizer=l2(weight_decay), activation="relu"))
# # model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(10,2), strides=(2, 2)))

# # print(model.layers[-1].output_shape)

# # model.add(Convolution2D(512, (10,2), 1, W_regularizer=l2(weight_decay), activation="relu"))
# # model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(10,2), strides=(2, 2)))

# # (bs, y, x, c) --> (bs, x, y, c)

# bs, x, y, c = model.layers[-1].output_shape
# model.add(Reshape((x, y*c)))

# model.add(Bidirectional(LSTM(256, return_sequences=False), merge_mode="concat"))
# model.add(Dense(5, activation="softmax"))


In [0]:
del(final_data_tr)
del(final_data_te)

NameError: ignored

In [0]:
tx = np.array(final_X_tr[:8000])
cvx = np.array(final_X_tr[8000:])
ty = np.array(final_labels_tr[:8000])
cvy = np.array(final_labels_tr[8000:])

In [0]:
del(final_labels_tr)
del(final_X_tr)

In [0]:
cvy = to_categorical(cvy)
ty = to_categorical(ty)

In [0]:
ty

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0002),
             metrics=["accuracy"])

In [0]:
model.fit(tx.reshape(-1,972,40,1), ty,
          batch_size=300,
          epochs=150,
          verbose=1,
          validation_data=(cvx.reshape(-1,972,40,1),cvy)
         )
          

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 50s 6ms/step - loss: 2.7217 - acc: 0.3967 - val_loss: 2.2547 - val_acc: 0.5225
Epoch 2/150
8000/8000 [==============================] - 39s 5ms/step - loss: 2.1181 - acc: 0.5423 - val_loss: 1.9594 - val_acc: 0.6125
Epoch 3/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.9386 - acc: 0.6085 - val_loss: 1.8529 - val_acc: 0.6335
Epoch 4/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.8319 - acc: 0.6515 - val_loss: 1.7270 - val_acc: 0.7255
Epoch 5/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.7674 - acc: 0.6744 - val_loss: 1.6737 - val_acc: 0.7420
Epoch 6/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.6836 - acc: 0.7088 - val_loss: 2.2373 - val_acc: 0.4930
Epoch 7/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.6260 - acc: 0.7295 - val_loss: 1.7249 - val_acc: 0.6860


KeyboardInterrupt: ignored

In [0]:
y_pred = model.predict(cvx.reshape(-1,972,40,1))
matrix = confusion_matrix(cvy.argmax(axis=1), y_pred.argmax(axis=1))

KeyboardInterrupt: ignored

In [0]:
matrix

array([[337,  21,  10,  24,   8],
       [ 17, 351,   6,   6,   3],
       [ 15,  19, 288,  66,   9],
       [ 22,  15,  34, 324,   6],
       [ 16,   0,  13,  43, 347]])

In [0]:
np.unique(ty, return_counts=True)

(array([0, 1, 2, 3, 4]), array([1600, 1617, 1603, 1599, 1581]))

In [0]:
np.unique(cvy, return_counts=True)
  

(array([0, 1, 2, 3, 4]), array([400, 383, 397, 401, 419]))